In [1]:
from supersecret import kenya_client_id, secret

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

# Your client credentials
#client_id = '<client_id>'
client_id = kenya_client_id
#client_secret = '<secret>'
client_secret = secret

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)

# Get token for the session
token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret)

# All requests using this session will have an access token automatically added
#resp = oauth.get("...")
#print(resp.content)

In [2]:
token

{'access_token': 'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE2OTk4ODYxNTAsImlhdCI6MTY5OTg4NTU1MCwianRpIjoiOWUyNDVlNTQtYTcxNy00YmQ2LWFkNjUtNzI2MWNlNmUxNzdlIiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZXJuaWN1cy5ldS9hdXRoL3JlYWxtcy9DRFNFIiwiYXVkIjoiQ0xPVURGRVJST19QVUJMSUMiLCJzdWIiOiJlNDlmYzVmMS02YjcyLTQxODEtODYzZC0wMzQwYWE5YzZhZjIiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJzaC03ZmU4MTgwYi00ZTQxLTQwZmQtYWQ4ZS1mMjRjYTAzZGE3MWEiLCJzY29wZSI6IkFVRElFTkNFX1BVQkxJQyBlbWFpbCBwcm9maWxlIHVzZXItY29udGV4dCIsImNsaWVudEhvc3QiOiIxNDEuNTEuMjQ5Ljg2IiwiZW1haWxfdmVyaWZpZWQiOmZhbHNlLCJjbGllbnRJZCI6InNoLTdmZTgxODBiLTRlNDEtNDBmZC1hZDhlLWYyNGNhMDNkYTcxYSIsIm9yZ2FuaXphdGlvbnMiOlsiZGVmYXVsdC03NGY3ZmI3Yy05NzU4LTQ3YzktODI0MS01ZmUxNDQ0NTAxNzQiXSwidXNlcl9jb250ZXh0X2lkIjoiZTM4ZjkyNWYtYmMyZC00NDkxLWFkYjYtMTQ0MzMyMGFiZGMzIiwiY29udGV4dF9yb2xlcyI6e30sImNvbnRleHRfZ3JvdXBzIjpbIi9hY2Nlc3NfZ3JvdXBzL3VzZXJfdHlwb2xvZ3kvY29wZXJuaWN1c19nZW5lcmFsLyIsIi9vcm

In [3]:
def sentinelhub_compliance_hook(response):
    response.raise_for_status()
    return response

oauth.register_compliance_hook("access_token_response", sentinelhub_compliance_hook)

In [7]:
# true color
evalscript = """
//VERSION=3
function setup() {
  return {
    input: ["B02", "B03", "B04"],
    output: { bands: 3 },
  }
}

function evaluatePixel(sample) {
  return [2.5 * sample.B04, 2.5 * sample.B03, 2.5 * sample.B02]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "bbox": [
               
                36.647443106,
                0.978438668,
                38.527295195,
                2.369927492
            ],
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": "2022-11-01T00:00:00Z",
                        "to": "2022-11-13T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 2500,
        "height": 2500,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request, headers={"Accept": "image/tiff"})

In [11]:
response

<Response [200]>

In [9]:
# Check if the request was successful
if response.status_code == 200:
    # Open a TIFF file in binary write mode and save the content of the response
    with open('output.tiff', 'wb') as f:
        f.write(response.content)
    print('TIFF file saved successfully.')
else:
    print('Failed to retrieve TIFF data. Status code:', response.status_code)

TIFF file saved successfully.


In [61]:
# all SL2A RAW DATA

#get token for the session:

token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret)

# query API:

evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: [
          "B01",
          "B02",
          "B03",
          "B04",
          "B05",
          "B06",
          "B07",
          "B08",
          "B8A",
          "B09",
          "B11",
          "B12",
        ],
        units: "DN",
      },
    ],
    output: {
      id: "default",
      bands: 12,
      sampleType: SampleType.UINT16,
    },
  }
}

function evaluatePixel(sample) {
  return [
    sample.B01,
    sample.B02,
    sample.B03,
    sample.B04,
    sample.B05,
    sample.B06,
    sample.B07,
    sample.B08,
    sample.B8A,
    sample.B09,
    sample.B11,
    sample.B12,
  ]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            36.647443106,
                            0.978438668,
                
                        ],
                        [
                            38.527295195,
                            0.978438668,
                        ],
                        [
                            38.527295195,
                            2.369927492
                        ],
                        [
                            36.647443106,
                            0.978438668,
                        ],
                        
                    ]
                ],
            },
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": "2022-10-01T00:00:00Z",
                        "to": "2022-10-31T00:00:00Z",
                    }
                },
                "processing": {"harmonizeValues": "true"},
            }
        ],
    },
    "output": {
        "width": 2500,
        "height": 2500,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/tiff"},
            }
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

In [62]:
#response.content

In [59]:
# Check if the request was successful
if response.status_code == 200:
    # Open a TIFF file in binary write mode and save the content of the response
    with open('output_raw.tiff', 'wb') as f:
        f.write(response.content)
    print('TIFF file saved successfully.')
else:
    print('Failed to retrieve TIFF data. Status code:', response.status_code)

Failed to retrieve TIFF data. Status code: 400


In [73]:
#NDVi GeoTIFF
#get token for the session:

token = oauth.fetch_token(token_url='https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token',
                          client_secret=client_secret)

# query API:

evalscript = """
//VERSION=3
function setup() {
  return {
    input: [
      {
        bands: ["B04", "B08"],
        units: "REFLECTANCE",
      },
    ],
    output: {
      id: "default",
      bands: 1,
      sampleType: SampleType.FLOAT32,
    },
  }
}

function evaluatePixel(sample) {
  let ndvi = (sample.B08 - sample.B04) / (sample.B08 + sample.B04)
  return [ndvi]
}
"""

request = {
    "input": {
        "bounds": {
            "properties": {"crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84"},
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                    [
                        [
                            36.647443106,
                            0.978438668,
                
                        ],
                        [
                            38.527295195,
                            0.978438668,
                        ],
                        [
                            38.527295195,
                            2.369927492
                        ],
                        [
                            36.647443106,
                            0.978438668,
                        ],
                        
                    ]
                ],
            },
        },
        "data": [
            {
                "type": "sentinel-2-l2a",
                "dataFilter": {
                    "timeRange": {
                        "from": "2022-10-01T00:00:00Z",
                        "to": "2022-10-31T00:00:00Z",
                    }
                },
                "processing": {"harmonizeValues": "true"},
            }
        ],
    },
    "output": {
        "width": 2500,
        "height": 2500,
        "responses": [
            {
                "identifier": "default",
                "format": {"type": "image/tiff"},
            }
        ],
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

In [74]:
#response.content

In [75]:
# Check if the request was successful
if response.status_code == 200:
    # Open a TIFF file in binary write mode and save the content of the response
    with open('output_NDVI.tiff', 'wb') as f:
        f.write(response.content)
    print('TIFF file saved successfully.')
else:
    print('Failed to retrieve TIFF data. Status code:', response.status_code)

TIFF file saved successfully.
